In [ ]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import json

import cv2
import torch
from tqdm import tqdm_notebook
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader
from torchvision import models

In [ ]:
META_DIR = "/kaggle/input/iwildcam2022-fgvc9/metadata/metadata/"
TRAIN_DIR = "/kaggle/input/iwildcam2022-fgvc9/train/train/"

train_data = json.load(open(META_DIR + 'iwildcam2022_train_annotations.json'))
test_data = json.load(open(META_DIR + 'iwildcam2022_test_information.json'))

In [ ]:
df_train = pd.DataFrame({'id': [item['id'] for item in train_data['annotations']],
                                'category_id': [item['category_id'] for item in train_data['annotations']],
                                'image_id': [item['image_id'] for item in train_data['annotations']],
                                'seq_id': [item['seq_id'] for item in train_data['images']],
                                'file_name': [item['file_name'] for item in train_data['images']],
                                'location': [item['location'] for item in train_data['images']],
                                'seq_num_frames': [item['seq_num_frames'] for item in train_data['images']],
                                'seq_frame_num': [item['seq_frame_num'] for item in train_data['images']]})



In [ ]:
# %%time
# import tqdm

# indices = []
# for i in tdqm(df_train['file_name']):
#     try:
#         Image.open(TRAIN_DIR + i)
#     except:        
#         print(i)
#         df_train.drop(df_train.loc[df_train['file_name']==i].index, inplace=True)

In [ ]:
seq_count = pd.read_csv(META_DIR + 'train_sequence_counts.csv')

In [ ]:
seq_count.head

In [ ]:
import torchvision

In [ ]:
import tqdm

In [ ]:
import torchvision.transforms as T
import torch.nn.functional as F

In [ ]:
# https://github.com/ultralytics/yolov3/blob/master/utils/datasets.py code for yolo dataset

In [ ]:
transforms = T.Compose(
           [
            T.ConvertImageDtype(torch.float32),
            T.Normalize(mean=0.5, std=0.5),  # map [0, 1] into [-1, 1]
            T.Resize(size=(1024, 1024)),
           ])

class CamDataset(Dataset):
    
    def __init__(self,
                 df,
                 images_dir,
                 seq_count,
                 img_size=1024
                ):
        self.df = df
        self.images_dir = images_dir
        self.seq_count = seq_count
        self.img_size = img_size
    
    def __len__(self):
        return self.seq_count.shape[0]
    
    def __getitem__(self, idx): 
        cur_idx_row = self.seq_count.iloc[idx]
        y = int(cur_idx_row['count'])
        img_rows = self.df.loc[self.df.seq_id == cur_idx_row['seq_id']]
        out = torch.zeros(1024,1024,3,img_rows.iloc[0].seq_num_frames)
        for index, img_row in img_rows.iterrows():
           img_path = os.path.join(self.images_dir, img_row['file_name'])
           
           out[:,:,:,img_row['seq_frame_num']] = transforms(torchvision.io.read_image(img_path)).permute(1,2,0)
        padded_out = F.pad(out, pad=(10 - out.shape[3], 0, 0, 0, 0, 0, 0, 0))
        return padded_out, y    

        
            
        

In [ ]:
# def collate_fn(data):
#     """
#        data: is a list of tuples with (example, label, length)
#              where 'example' is a tensor of arbitrary shape
#              and label/length are scalars
#     """
#     examples, labels = zip(*data)
#     F.pad(source, pad=(0, 0, 0, 70 - source.shape[0])

#     return features.float(), labels.long(), lengths.long()

In [ ]:
from torch.utils.data import DataLoader
training_data = CamDataset(df_train, TRAIN_DIR, seq_count)
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)


In [ ]:
%%time
train_features, train_labels = next(iter(train_dataloader))

In [ ]:
!pip install cython; pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!pip install -r '/kaggle/input/centertrack/requirements.txt'

In [ ]:
!cd /kaggle/input/centertrack/src/lib/model/networks/
!git clone https://github.com/CharlesShang/DCNv2/ # clone if it is not automatically downloaded by `--recursive`.
!cd DCNv2
!./make.sh

In [ ]:
%tb
import sys
CENTERTRACK_PATH = '/kaggle/input/centertrack/src/lib/'
sys.path.insert(0, CENTERTRACK_PATH)

from detector import Detector
from opts import opts

MODEL_PATH = '/kaggle/input/centertrackmodel/coco_tracking.pth'
TASK = 'tracking,ddd' # or 'tracking,multi_pose' for pose tracking and 'tracking,ddd' for monocular 3d tracking
opt = opts().init('{} --load_model {}'.format(TASK, MODEL_PATH).split(' '))
detector = Detector(opt)



In [ ]:
images = ['''image read from open cv or from a video''']
for img in images:
  ret = detector.run(img)['results']